In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from scipy import optimize
import matplotlib 

%matplotlib inline 

//anaconda/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [6]:
asset_ret= pd.read_excel( 'ret_cov.xlsx', sheet_name= 'Return', index_col= 0, header=None).astype(float)
asset_ret= asset_ret.iloc[:, 0]
asset_ret.name= 'Return'
asset_ret.index.name= 'Asset'
asset_cov= pd.read_excel( 'ret_cov.xlsx', sheet_name= 'Cov', index_col=0, header= 0).astype(float)

In [7]:
asset_ret

Asset
EEM           -0.260481
EFA           -0.169746
HYG           -0.076114
USO            0.311453
IAU            0.012500
SOYB           0.159935
VNQ           -0.244119
equity_bask   -0.215087
Name: Return, dtype: float64

In [8]:
asset_cov

,EEM,EFA,HYG,USO,IAU,SOYB,VNQ,equity_bask
Asset,,,,,,,,
EEM,0.020728,0.010967,0.003591,0.002934,0.007230,-0.004031,0.010706,0.009355
EFA,0.010967,0.011653,0.002525,0.015091,-0.000635,-0.005479,0.010187,0.013283
HYG,0.003591,0.002525,0.001568,-0.001790,-0.000164,-0.000643,0.006164,0.002907
USO,0.002934,0.015091,-0.001790,0.087967,-0.001554,-0.003753,-0.007435,0.020453
IAU,0.007230,-0.000635,-0.000164,-0.001554,0.013980,-0.001150,-0.007282,-0.005549
SOYB,-0.004031,-0.005479,-0.000643,-0.003753,-0.001150,0.015531,-0.006119,-0.008038
VNQ,0.010706,0.010187,0.006164,-0.007435,-0.007282,-0.006119,0.044217,0.018621
equity_bask,0.009355,0.013283,0.002907,0.020453,-0.005549,-0.008038,0.018621,0.019806


In [10]:
asset_vol= np.sqrt(np.diag( asset_cov))
asset_vol= pd.Series( asset_vol, index= asset_cov.index)
asset_vol

Asset
EEM            0.143972
EFA            0.107951
HYG            0.039602
USO            0.296592
IAU            0.118238
SOYB           0.124622
VNQ            0.210279
equity_bask    0.140733
dtype: float64

In [14]:
### optimization
### min -1* ret
### Constrain: 1. volatility<= risk budget (5% Annual)
### 2. fully invested 
### 3. long only 
### 4. no asset weight>= 40%


def obj_func (weight): 
    '''
    weight: pd Series, asset weight
    
    RETURN: float, Portfolio Return (Minus)
    '''
    return -1* (weight*asset_ret).sum()

def obj_func_jac ( weight): 
    '''
    weight: pd Series, asset weight
    
    RETURN: np array, obj_func derivatives, 
    '''
    
    return -1* asset_ret.values.flatten()

def constr_0 (weight):
    '''
    weight: pd Series, asset weight
    
    RETURN: float, portfolio volatility (Annual)
    '''
    
    return 0.05- np.sqrt( np.dot( weight, np.dot( asset_cov, weight)))


def constr_0_jac( weight):
    '''
    weight: pd Series, asset weight, 
    cov: pd df, asset cov (Annual)
    
    RETURN: np array, constr_0 derivatives
    '''
    
    return  -1* np.dot( asset_cov, weight).flatten()



def constr_1( weight ):
    '''
    weight: pd Series, weight
    
    RETURN: float, weight sum
    '''
    
    return weight.sum()-1

def constr_1_jac( weight): 
    '''
    weight: pd Series, weight
    
    RETURN: np array, weight derivative
    '''
    
    return np.ones( weight.shape[0]).flatten()

In [19]:
bound_low= 0
bound_high= 0.4

opt= optimize.minimize( fun= obj_func, jac= obj_func_jac, 
                       x0 = np.ones( asset_cov.shape[0]).flatten()* 1/asset_cov.shape[0] ,
                       constraints= ({'type': 'ineq', 'fun': constr_0, 'jac': constr_0_jac}, 
                                    {'type': 'eq', 'fun': constr_1, 'jac': constr_1_jac}),
                       method='SLSQP',
                       bounds= [[bound_low, bound_high]]* asset_cov.shape[0],
                       options= {'disp':True}
                      )

Iteration limit exceeded    (Exit mode 9)
            Current function value: -0.05628198332401529
            Iterations: 101
            Function evaluations: 217
            Gradient evaluations: 101


In [21]:
np.linalg.det(asset_cov)

5.707578636699416e-18